TP1

Laboratorio de Datos

1er Cuatrimestre - 2024

Iván Exequiel Pintos - Joaquín Rovner - Juan José García Vizioli

- Importamos todas las librerías que venimos usando:

In [2]:
import seaborn as sns
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn.objects as so

from scipy import stats
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
#from formulaic import model_matrix

- Leemos el dataset:

In [3]:
#@title Usando Colab

from google.colab import drive
drive.mount('/content/drive')

sube_2023_file_path = '/content/drive/My Drive/LDD-TP1/sube-2023.csv'

Mounted at /content/drive


In [ ]:
#@title Usando Jupyter

sube_2023_file_path = '../LDD-TP1/sube-2023.csv'

In [4]:
SUBE_2023 = pd.read_csv(sube_2023_file_path)

#Procesamiento de Datos

##1)

###a)

Visualizamos el tipo de dato de cada columna:

In [ ]:
def ver_tipos_datos_columnas():
  for i in SUBE_2023:
      print(i,'\t',type(SUBE_2023[i][0]))

ver_tipos_datos_columnas()

DIA_TRANSPORTE 	 <class 'str'>
NOMBRE_EMPRESA 	 <class 'str'>
LINEA 	 <class 'str'>
AMBA 	 <class 'str'>
TIPO_TRANSPORTE 	 <class 'str'>
JURISDICCION 	 <class 'str'>
PROVINCIA 	 <class 'str'>
MUNICIPIO 	 <class 'str'>
CANTIDAD 	 <class 'numpy.int64'>
DATO_PRELIMINAR 	 <class 'str'>


Le cambiamos el formato a la columna DIA_TRANSPORTE:

In [5]:
SUBE_2023['DIA_TRANSPORTE'] = pd.to_datetime(SUBE_2023['DIA_TRANSPORTE']) #investigar argumento "format"

Corroboramos que el tipo de dato de la columna ahora es un time_stamp:

In [ ]:
ver_tipos_datos_columnas()

DIA_TRANSPORTE 	 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
NOMBRE_EMPRESA 	 <class 'str'>
LINEA 	 <class 'str'>
AMBA 	 <class 'str'>
TIPO_TRANSPORTE 	 <class 'str'>
JURISDICCION 	 <class 'str'>
PROVINCIA 	 <class 'str'>
MUNICIPIO 	 <class 'str'>
CANTIDAD 	 <class 'numpy.int64'>
DATO_PRELIMINAR 	 <class 'str'>


###b)

####i)

Agregamos la columna FECHA_DIA a partir de DIA_TRANSPORTE

In [6]:
SUBE_2023["FECHA_DIA"] = SUBE_2023['DIA_TRANSPORTE'].apply(lambda x : x.strftime('%a'))

Vemos que la columna sigue el formato de 3 letras en inglés pedido

In [ ]:
SUBE_2023.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR,FECHA_DIA
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO,Sun
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO,Sun
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO,Sun
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO,Sun
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO,Sun


####ii)


Agregamos la columna FECHA_ORDINAL a partir de DIA_TRANSPORTE

In [7]:
SUBE_2023["FECHA_ORDINAL"] = SUBE_2023['DIA_TRANSPORTE'].apply(lambda x : int(x.strftime('%d'))) #Da el día del mes de 1 a 31. Si quisiéramos ir de 1 a 366, usamos '%-j'.

Vemos que la columna FECHA_ORDINAL tiene el formato pedido y que además sus datos son int:

In [ ]:
display(SUBE_2023.head())
print('Tipo de dato de FECHA_ORDINAL:', SUBE_2023['FECHA_ORDINAL'].dtype)

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR,FECHA_DIA,FECHA_ORDINAL
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO,Sun,1
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO,Sun,1
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO,Sun,1
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO,Sun,1
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO,Sun,1


Tipo de dato de FECHA_ORDINAL: int64


####iii)

Agregamos la columna FECHA_MES a partir de DIA_TRANSPORTE

In [8]:
SUBE_2023["FECHA_MES"] = SUBE_2023['DIA_TRANSPORTE'].apply(lambda x : x.strftime('%m'))

Vemos que la columna sigue el formato de string de número de 2 dígitos pedido

In [ ]:
SUBE_2023.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR,FECHA_DIA,FECHA_ORDINAL,FECHA_MES
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO,Sun,1,01
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO,Sun,1,01
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO,Sun,1,01
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO,Sun,1,01
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO,Sun,1,01


##2)

Texto 2)

In [ ]:
datos_amba = SUBE_2023[np.logical_and(SUBE_2023['AMBA'] == 'SI', SUBE_2023['DATO_PRELIMINAR'] == 'NO')]
columnas_datos_amba = ['DIA_TRANSPORTE','FECHA_DIA','FECHA_MES','FECHA_ORDINAL','JURISDICCION','LINEA','CANTIDAD','TIPO_TRANSPORTE']
datos_amba = datos_amba[columnas_datos_amba].rename(columns = {'DIA_TRANSPORTE' : 'FECHA', 'CANTIDAD' : 'PASAJEROS'}).rename(columns=str.lower).reset_index().drop(["index"], axis = 1)#JOACO METIO MANO ACA PARA RESETEAR INDICES
datos_amba.head()

: 

##3)

###a)

Texto 3)a)

In [24]:
datos_amba['tipo_transporte'].unique() #Ver qué tipos de transporte hay

array(['COLECTIVO', 'TREN', 'SUBTE'], dtype=object)

In [32]:
total_pasajeros_amba = datos_amba['pasajeros'].sum()

datos_amba_colectivo = datos_amba[datos_amba['tipo_transporte'] == 'COLECTIVO']
datos_amba_tren = datos_amba[datos_amba['tipo_transporte'] == 'TREN']
datos_amba_subte = datos_amba[datos_amba['tipo_transporte'] == 'SUBTE']

pasajeros_colectivo_amba = datos_amba_colectivo['pasajeros'].sum()
pasajeros_tren_amba = datos_amba_tren['pasajeros'].sum()
pasajeros_subte_amba = datos_amba_subte['pasajeros'].sum()

porcentaje_pasajeros_colectivo = pasajeros_colectivo_amba * 100 / total_pasajeros_amba
porcentaje_pasajeros_tren = pasajeros_tren_amba * 100 / total_pasajeros_amba
porcentaje_pasajeros_subte = pasajeros_subte_amba * 100 / total_pasajeros_amba

print('Porcentaje anual en colectivo:', str(round(porcentaje_pasajeros_colectivo, 2)) + '%')
print('Porcentaje anual en tren:', str(round(porcentaje_pasajeros_tren, 2)) + '%')
print('Porcentaje anual en subte:', str(round(porcentaje_pasajeros_subte, 2)) + '%')

Porcentaje anual en colectivo: 83.76%
Porcentaje anual en tren: 9.43%
Porcentaje anual en subte: 6.81%


###b)

Texto 3)b)

In [50]:
fila_record_subte_amba = datos_amba_subte.nlargest(1, 'pasajeros').iloc[0]

record_subte = mes_record_subte, linea_record_subte = fila_record_subte_amba['fecha_mes'], fila_record_subte_amba['linea']

print('Mes y linea con mayor cantidad de pasajeros en subte:', record_subte)

Mes y linea con mayor cantidad de pasajeros en subte: ('12', 'LINEA_B')


###c)

Texto 3)c)

In [67]:
dia_habil_menor_std = datos_amba.groupby('fecha_dia')['pasajeros'].std().drop(['Sun','Sat']).idxmin()
dia_habil_menor_std

'Mon'

In [ ]:
subtes = datos_amba[datos_amba["tipo_transporte"]=="SUBTE"].reset_index().drop(["index"], axis = 1) #CREO A PARTIR DE AMBA UN DATAFRAME DE SUBTE PARA ESTUDIARLO
subtes = subtes[subtes["linea"] != "LIN_PREMETRO"] #EXCLUIMOS PREMETRO POR LA ESCASES DE DATOS EN COMPARACION A LAS OTRAS LINEAS
subtes["linea"].replace({"LINEA_A": "LINEA SUBTE A", "LINEA_B": "LINEA SUBTE B"}, inplace = True)


promedio_por_mes = subtes.groupby('fecha_mes')['pasajeros'].sum() / subtes['linea'].nunique()
promedio_subtes = pd.DataFrame({'fecha_mes': promedio_por_mes.index, 'promedio': promedio_por_mes}).drop(["fecha_mes"], axis = 1)
subtes = pd.merge(subtes, promedio_subtes, on='fecha_mes')


subtes

In [ ]:
#GRAICOS SUELTOS COMO PARA HACER ALGO
df_agrupado = SUBE_2023.groupby('AMBA')['CANTIDAD'].sum().reset_index()
(
    so.Plot(data = df_agrupado, x = "AMBA", y = "CANTIDAD")
    .add(so.Bar())
)

In [ ]:
amba_agrupados = datos_amba.groupby('tipo_transporte')['pasajeros'].sum().reset_index()
(
    so.Plot(data = amba_agrupados, x = "tipo_transporte", y = "pasajeros")
    .add(so.Bar())
)

In [ ]:
#GRAFICO PASAJEROS POR LINEA

#MODIFICAMOS COLORES SEGUN LINEA
linea_colores = {
    'LINEA SUBTE A': '#0fb7d9', # azul
    'LINEA SUBTE B': '#d60b15', # azul
    'LINEA SUBTE C': '#0220c7', # azul
    'LINEA SUBTE D': '#075202', # azul
    'LINEA SUBTE E': '#23096b', # azul
    'LINEA SUBTE H': '#e3dd22', # azul
}



(
    so.Plot(data = subtes, x = "fecha_mes", y = "pasajeros", color="linea") #REWVISAR COLORES LINEAS PARA MATCHEAR CON COLORES ORIGINALES
    .add(so.Line(color = "#000000"), x = "fecha_mes", y = "promedio",  label = "PROMEDIO" ) #PROMEDIO ENTRE TODAS LAS LINEAS
    .add(so.Dot(color = "#000000"), x = "fecha_mes", y = "promedio") #PROMEDIO ENTRE TODAS LAS LINEAS
    .add(so.Dot(), so.Agg("sum"))
    .add(so.Line(), so.Agg("sum"))
    .scale(color=linea_colores)
    .label(x="Meses", y="Cantidad de Pasajeros", title="Pasajeros por linea de subte 2023")
)

#AUMENTA EL USO DEL SUBTE EN MARZO POR FINALIZACION DE LA EPOCA VACACIONAL
#CAIDA LINEA D EN DICIEMBRE PUEDE RELACIONARES CON LAS OBRAS QUE ESTABAN POR COMENZAR
#CAIDA LINEA A EN MARZO/ABRIL SE DEBE A REFORMAS EN CIERTAS ESTACIONES DE LA MISMA